# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [1]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [2]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=10_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

(10000, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

#### Análise de Target

In [3]:
df['loan_status'].value_counts(dropna=False, normalize=True) * 100

loan_status
Fully Paid            71.89
Charged Off           17.13
Current               10.24
Late (31-120 days)     0.58
In Grace Period        0.12
Late (16-30 days)      0.04
Name: proportion, dtype: float64

In [4]:
TARGET = 'target_risco_credito'

status_de_risco = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    #"Late (16-30 days)"
]
df[TARGET] = df["loan_status"].isin(status_de_risco).astype(int)

df[TARGET].value_counts(dropna=False, normalize=True) * 100

target_risco_credito
0    82.29
1    17.71
Name: proportion, dtype: float64

### Safras

In [5]:
#date_col=['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d','next_pymnt_d']

temporal_columns = [
    "issue_d",                      # Data de emissão do empréstimo
    "earliest_cr_line",             # Primeira linha de crédito do cliente
    "last_pymnt_d",                 # Último pagamento realizado
    "last_credit_pull_d",           # Última consulta ao crédito
    "next_pymnt_d",                 # Próximo pagamento previsto (se aplicável)
    #"last_pymnt_amnt",             # (associada à data de pagamento)
    "debt_settlement_flag_date",    # Data em que houve acordo de dívida
    "settlement_date"               # Data em que o acordo foi fechado
]


# tratamento para colunas de data
for col in temporal_columns:
    df[col] = pd.to_datetime(df[col], format="%b-%Y")


df["safra"] = df["issue_d"].dt.to_period("M")#.astype(str)  # formato 'YYYY-MM'

print(df["safra"].min())
print(df["safra"].max())

2015-12
2015-12


### Captura Inicial

In [6]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    target_col=TARGET,
    default_keys=["id"]
)

trail.take_snapshot(df, name="amostra")

In [7]:
trail.describe_snapshot("amostra")


📄 Descrição do snapshot 'amostra':

▶️ Shape: (10000, 153)
▶️ Chaves de duplicação: ['id']
   • Duplicatas nas chaves: 0

🧱 Tipos de dados:


id                           int64
member_id                  float64
loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
                           ...    
settlement_amount          float64
settlement_percentage      float64
settlement_term            float64
target_risco_credito         int32
safra                    period[M]
Length: 153, dtype: object


🔎 Colunas detectadas automaticamente:
   • Numéricas (89): ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'char

member_id                                     10000
revol_bal_joint                               10000
sec_app_fico_range_high                       10000
sec_app_earliest_cr_line                      10000
sec_app_inq_last_6mths                        10000
sec_app_mort_acc                              10000
sec_app_open_acc                              10000
sec_app_revol_util                            10000
sec_app_open_act_il                           10000
sec_app_num_rev_accts                         10000
sec_app_chargeoff_within_12_mths              10000
sec_app_collections_12_mths_ex_med            10000
sec_app_mths_since_last_major_derog           10000
sec_app_fico_range_low                        10000
desc                                           9999
dti_joint                                      9935
verification_status_joint                      9935
annual_inc_joint                               9935
orig_projected_additional_accrued_interest     9934
hardship_typ


📊 Estatísticas numéricas:


,count,mean,std,min,25%,50%,75%,max
id,10000.0,6.804055e+07,1.974650e+06,361774.00,6.788920e+07,68242559.50,6.847419e+07,68617057.00
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,10000.0,1.510444e+04,8.717528e+03,1000.00,8.000000e+03,14000.00,2.000000e+04,35000.00
funded_amnt,10000.0,1.510444e+04,8.717528e+03,1000.00,8.000000e+03,14000.00,2.000000e+04,35000.00
funded_amnt_inv,10000.0,1.510090e+04,8.714505e+03,1000.00,8.000000e+03,14000.00,2.000000e+04,35000.00
...,...,...,...,...,...,...,...,...
hardship_last_payment_amount,84.0,1.836843e+02,1.963978e+02,0.06,4.144500e+01,102.38,2.620550e+02,780.05
settlement_amount,275.0,4.903284e+03,3.525341e+03,250.00,2.065930e+03,4143.57,7.148730e+03,17500.00
settlement_percentage,275.0,4.703724e+01,5.083315e+00,30.00,4.500000e+01,45.00,5.000000e+01,65.04
settlement_term,275.0,1.353455e+01,7.460461e+00,0.00,8.000000e+00,14.00,1.800000e+01,24.00



🏷️ Estatísticas categóricas:


,count,unique,top,freq
term,10000,2,36 months,7052
grade,10000,7,B,3000
sub_grade,10000,35,C1,682
emp_title,9443,5622,Teacher,250
emp_length,9451,11,10+ years,3333
home_ownership,10000,3,MORTGAGE,4925
verification_status,10000,3,Source Verified,3589
loan_status,10000,6,Fully Paid,7189
pymnt_plan,10000,2,n,9999
url,10000,10000,https://lendingclub.com/browse/loanDetail.acti...,1



📈 Histogramas (categorias apenas):
  term: 2 valores distintos (top 3: {' 36 months': 7052, ' 60 months': 2948})
  grade: 7 valores distintos (top 3: {'B': 3000, 'C': 2873, 'A': 1824})
  sub_grade: 35 valores distintos (top 3: {'C1': 682, 'B4': 648, 'B2': 616})
  emp_length: 12 valores distintos (top 3: {'10+ years': 3333, '< 1 year': 941, '2 years': 872})
  home_ownership: 3 valores distintos (top 3: {'MORTGAGE': 4925, 'RENT': 3934, 'OWN': 1141})
  verification_status: 3 valores distintos (top 3: {'Source Verified': 3589, 'Not Verified': 3540, 'Verified': 2871})
  loan_status: 6 valores distintos (top 3: {'Fully Paid': 7189, 'Charged Off': 1713, 'Current': 1024})
  pymnt_plan: 2 valores distintos (top 3: {'n': 9999, 'y': 1})
  purpose: 12 valores distintos (top 3: {'debt_consolidation': 5724, 'credit_card': 2472, 'home_improvement': 561})
  title: 13 valores distintos (top 3: {'Debt consolidation': 5658, 'Credit card refinancing': 2443, 'Home improvement': 546})
  addr_state: 49 valo

#### Limpeza com Vassoura

In [8]:
assert set(df[TARGET].dropna().unique()) == {0, 1}, "TARGET não é binário!"

In [9]:
vs = Vassoura(
    df,
    id_cols = ['id','member_id'],
    date_cols=['safra'],
    ignore_cols=['url']+temporal_columns,
    drop_ignored=True,
    target_col=TARGET,
    verbose='basic',
    
    engine='polars',
    
    adaptive_sampling=True,

    heuristics=[
        'missing',
        'variance',
        "iv",
        "graph_cut",
        "corr",
        "vif",
        'ks_separation',
        #"psi_stability",
        #"perm_importance_lgbm",
        #"partial_corr_cluster",
        #"drift_vs_target_leakage",
    ],
    
    thresholds={
        "missing": 0.60,
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,
    },
    n_steps=2,         # controla quantas remoções por iteração
    vif_n_steps=2      # VIF será sempre 1-step (a menos que você altere)
)

df_limpo = vs.run(recompute=True)

[Vassoura] Missing heuristic (thr>0.6)
  → dropped ['desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'settlement_status', 'settlement_amount', 'settlement_percentage', 'settlemen

INFO | Tipos detectados: 47 numéricas, 10 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 47 variáveis numéricas (engine=pandas)


  → dropped ['last_fico_range_high'] (corr>0.85)
  → dropped ['tot_hi_cred_lim'] (corr>0.85)
  → dropped ['total_rev_hi_lim'] (corr>0.85)


INFO | Tipos detectados: 44 numéricas, 10 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 44 variáveis numéricas (engine=pandas)


  → dropped ['bc_util'] (corr>0.85)
  → dropped ['mths_since_rcnt_il'] (corr>0.85)


...

  → dropped ['annual_inc'] (vif>5)


INFO | Tipos detectados: 20 numéricas, 10 categóricas
INFO | Desconsiderando 1170 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 20 variáveis


  → dropped ['num_rev_tl_bal_gt_0'] (vif>5)


INFO | Tipos detectados: 19 numéricas, 10 categóricas
INFO | Desconsiderando 1170 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 19 variáveis
INFO | Tipos detectados: 17 numéricas, 8 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 17 variáveis numéricas (engine=pandas)


  → dropped ['emp_title', 'emp_length', 'zip_code', 'addr_state', 'revol_bal', 'total_bal_il'] (ks<0.05)


In [10]:
df_limpo.head()

,id,member_id,safra,target_risco_credito,term,grade,sub_grade,home_ownership,verification_status,loan_status,purpose,title,dti,inq_last_6mths,out_prncp_inv,collection_recovery_fee,last_pymnt_amnt,open_acc_6m,max_bal_bc,inq_fi,inq_last_12m,avg_cur_bal,bc_open_to_buy,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,percent_bc_gt_75
0,361774,NaN,2015-12,0,36 months,B,B1,MORTGAGE,Not Verified,Fully Paid,debt_consolidation,Debt consolidation,5.10,0.0,0.0,0.0,586.68,1.0,5634.0,0.0,2.0,49598.0,832.0,52.0,3.0,4.0,52.0,3.0,66.7
1,577926,NaN,2015-12,0,36 months,E,E3,RENT,Source Verified,Fully Paid,home_improvement,Home improvement,32.12,2.0,0.0,0.0,5620.34,4.0,2183.0,3.0,5.0,3031.0,7405.0,4.0,4.0,0.0,17.0,4.0,0.0
2,699575,NaN,2015-12,0,36 months,C,C2,MORTGAGE,Not Verified,Fully Paid,debt_consolidation,Debt consolidation,13.12,0.0,0.0,0.0,19906.26,2.0,5826.0,9.0,15.0,11551.0,35864.0,4.0,4.0,1.0,4.0,0.0,0.0
3,811529,NaN,2015-12,0,36 months,A,A4,RENT,Not Verified,Fully Paid,credit_card,Credit card refinancing,7.98,0.0,0.0,0.0,1553.29,2.0,3051.0,0.0,1.0,3085.0,38769.0,4.0,4.0,0.0,4.0,9.0,0.0
4,849795,NaN,2015-12,0,36 months,A,A2,MORTGAGE,Source Verified,Fully Paid,credit_card,Credit card refinancing,8.36,0.0,0.0,0.0,11496.70,0.0,2742.0,2.0,1.0,99056.0,58.0,19.0,19.0,4.0,19.0,6.0,100.0


In [ ]:
vs.generate_report("../reports/vassoura_report.html")

INFO | Tipos detectados: 115 numéricas, 27 categóricas
